### 데이터 로딩과 전처리

In [1]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [36]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head(10)

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [3]:
#유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외
ratings = ratings[ratings['ratings']>=3]

In [4]:
ratings.head()
filtered_data_size = len(ratings)

In [5]:
print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [6]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [7]:
ratings['counts']  

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [37]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head(20)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [9]:
movies

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [11]:
import re
movies['title'] = movies['title'].str.lower() # 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
movies['title'] = movies['title'].str.replace(pat=r'[[\d]]', repl=r'', regex=True)



movies.head(20)

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy
5,6,heat (1995),Action|Crime|Thriller
6,7,sabrina (1995),Comedy|Romance
7,8,tom and huck (1995),Adventure|Children's
8,9,sudden death (1995),Action
9,10,goldeneye (1995),Action|Adventure|Thriller


### 데이터탐색

In [12]:
# 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [13]:
# 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [15]:
#가장 인기 있는 영화 30개(인기순)
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)


movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [16]:
#내가 선호하는 영화를 5가지 ratings에 추가

# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = [ '2000' , '2022' ,'2460' ,'3211' ,'1941']

# '9000'이라는 user_id가 위 영화들의 평점을 5로 주었다고 가정하겠습니다.

my_playlist = pd.DataFrame({'user_id': ['9000']*5, 'movie_id': my_favorite, 'counts':[5]*5, 'timestamp':[0]*5})               

if not ratings.isin({'user_id':['9000']})['user_id'].any():  # user_id에 '9000'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.


,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569
0,9000,2000,5,0
1,9000,2022,5,0
2,9000,2460,5,0
3,9000,3211,5,0
4,9000,1941,5,0


In [17]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}


In [18]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['9000'])    
print(movie_to_idx['2000'])

6039
3628


In [21]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings



user_id column indexing Fail!!
movie column indexing Fail!!


,user_id,movie_id,counts,timestamp
0,0,0,5,978300760
1,0,1,3,978302109
2,0,2,3,978301968
3,0,3,4,978300275
4,0,4,5,978824291
...,...,...,...,...
0,6039,3628,5,0
1,6039,3629,5,0
2,6039,3630,5,0
3,6039,3631,5,0


### Matrix Factorization(MF)

In [22]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data


<6040x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [23]:
#ratings=ratings.astype('int64')

In [24]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)


In [25]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose


<3633x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [26]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [42]:
#my favorite 영화 중 하나인 movie_id 2000 에 대한 나의 선호도 
user, movie = user_to_idx['9000'], movie_to_idx['2000']
user_vector, movie_vector = als_model.user_factors[user], als_model.item_factors[movie]



In [43]:
user_vector

array([ 0.00815141, -0.00183978, -0.00120355, -0.00759519,  0.00854771,
       -0.00854142, -0.00064708, -0.00625331, -0.00489773, -0.00527086,
        0.00066041, -0.00457463, -0.00291316, -0.01236839,  0.00215313,
       -0.00174545,  0.00415454,  0.0024123 ,  0.00252105, -0.01048139,
        0.00240443,  0.00903332, -0.00192822,  0.00747692,  0.01111463,
       -0.00608839, -0.0056766 ,  0.00804794, -0.00320271, -0.00069974,
       -0.0056592 ,  0.01014453, -0.00112962, -0.00055617,  0.01373417,
       -0.00478972, -0.01526403, -0.00523746, -0.01421776,  0.00904112,
       -0.00554188,  0.00881699,  0.01103814, -0.01040141, -0.0017459 ,
       -0.00691141,  0.00093469, -0.01021775,  0.00032158,  0.01098759,
       -0.0110099 , -0.00502037,  0.01208709, -0.00237487, -0.00859987,
       -0.00156761, -0.0031841 ,  0.00448824, -0.00630796,  0.0041038 ,
       -0.00181575,  0.01242738, -0.00774275, -0.0020182 ,  0.00049727,
       -0.00352559,  0.01145351,  0.00847617,  0.00298669, -0.00

In [44]:
movie_vector

array([ 4.97819565e-05,  2.32352795e-05,  2.71574499e-05,  2.08806432e-06,
        7.60444600e-05, -7.24028769e-06,  3.45013541e-05,  1.19554252e-05,
        8.73868339e-06,  5.19556925e-05,  5.25977230e-05, -9.19552633e-07,
        3.41698542e-05,  1.60330874e-05,  5.24891693e-05,  5.53031969e-05,
        8.77717466e-05,  6.06424401e-05,  7.20590324e-05, -1.39766671e-05,
        7.30589454e-05,  9.47194494e-05,  4.53077446e-05,  7.71599152e-05,
        1.47914820e-04,  8.70478652e-06,  4.17813499e-05,  7.71401465e-05,
        1.39773601e-05,  5.23007329e-05,  4.95272470e-05,  9.23228872e-05,
        2.38895755e-05,  6.23861124e-05,  1.04301711e-04,  2.53463113e-05,
       -3.55660195e-05,  4.19814896e-05, -4.38067618e-05,  1.10818910e-04,
        4.90999337e-05,  7.72718922e-05,  6.69961810e-05,  2.70967757e-06,
        1.95832617e-05,  2.25528129e-05,  3.16970436e-05,  1.20669638e-05,
        4.62577700e-05,  1.12130278e-04, -1.90584815e-05,  1.67623075e-05,
        1.32155532e-04,  

In [45]:
# 두 벡터값을 내적하는 코드
np.dot(user_vector, movie_vector)

2.525628e-05

In [48]:
# movie_id 1 인 Toy Story 와 비슷한 영화 추천 

favorite_movie = 1
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(40, 1.0000002),
 (50, 0.7756828),
 (322, 0.5972231),
 (33, 0.5679747),
 (4, 0.5111459),
 (110, 0.49879003),
 (330, 0.456437),
 (20, 0.42835295),
 (10, 0.41498417),
 (255, 0.39437842),
 (545, 0.3418922),
 (34, 0.3376242),
 (458, 0.33710006),
 (277, 0.3295726),
 (16, 0.3284639)]

In [52]:
#movie_to_idx 를 뒤집어, index로부터 movie_id를 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[1,
 3114,
 34,
 588,
 2355,
 1265,
 364,
 2321,
 595,
 1923,
 317,
 1907,
 367,
 2384,
 2687]

In [53]:
#movie_recommended = als_model.recommend(6039, csr_data, N=20, filter_already_liked_items=True)
#movie_recommended


user = user_to_idx['9000']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(399, 0.002381776),
 (41, 0.0022827263),
 (470, 0.0021239405),
 (2273, 0.0021029767),
 (2609, 0.0020542124),
 (1003, 0.0019961228),
 (2229, 0.0018909986),
 (2381, 0.0018892334),
 (1843, 0.0018817518),
 (2593, 0.001878903),
 (2095, 0.0018684772),
 (811, 0.0018616124),
 (2376, 0.0018562286),
 (1854, 0.0018452726),
 (2661, 0.0018316892),
 (224, 0.0018099546),
 (3309, 0.0017708614),
 (107, 0.0017414283),
 (160, 0.0017329403),
 (3050, 0.0017109867)]

In [54]:
[idx_to_movie[i[0]] for i in movie_recommended]

[3155,
 1961,
 1060,
 1900,
 725,
 1772,
 1415,
 2594,
 2670,
 621,
 3384,
 2763,
 2621,
 113,
 635,
 1617,
 1504,
 480,
 356,
 1056]

### 회고: 
movie_id 대신 title 로 모델을 돌리고 싶어서 시도했다가 title 데이터에 있는 괄호() 와 숫자 등 이 포함되어 있다는 것을 발견하고 
    정규식으로 전처리를 하고 싶었다. 소문자로 먼저 바꾼 다음 [^a-z]으로 전처리를 시도 했으나 아무리 해도 ()를 없애지 못하였고 그대로 진행하니 결국은 에러가 나서 진행이 불가능했다. 
    그래서.... movie_id 로 실습을 계속 할 수 밖에 없어서 마음에 들지 않고 내 실력의 한계를 느껴 답답한 마음만 들며 마무리 지었다. 
    요즘은 시간이 없어서 넷플릭스를 못보고 있지만 한 때 매일 넷플릭스를 끼고 살았던 시절을 회상하며 넷플릭스가 나에게 추천해 주었던 수많은 영화를 생각하며 이번 노드를 공부하니 재미는 있었다. (문제는 내 실력... ㅠ)
    